서울시 부동산 실거래가 정보 맵 표현

In [74]:
import requests as req
import json
import pandas as pd
from tqdm import tqdm # 진행 상황을 나타내는 라이브러리

import folium # 지도 시각화 라이브러리
from folium import Marker
import googlemaps # Google Maps API 라이브러리

pd.options.display.max_rows = 50000


class AptPrice:

    def __init__(self, page_number):
        self.page_number = page_number

    def get_apt_data(self):
        # 서울시 데이터에서 아파트 실거래가 정보를 가져옴.
        seoul_apt = []
        for i in tqdm(range(1, self.page_number)): # 페이지 수 만큼 반복
            # API URL을 생성하고 요청을 보냄
            url = f"http://openapi.seoul.go.kr:8088/6364595a4b73736435336944534355/json/tbLnOpendataRtmsV/{(i-1)*1000+1}/{i*1000}/2022/%20/%20/%20/%20/%20/%20/%20/%20/%20/%EC%95%84%ED%8C%8C%ED%8A%B8/"
            response = req.get(url)
            response_js = response.json() # JSON 데이터로 변환
            row_data = response_js['tbLnOpendataRtmsV']['row'] # 필요한 데이터 추출
            for row in row_data:
                data = {"자치구명":row['SGG_NM'],
                        "건물명":row['BLDG_NM'],
                        "매매가(만원)":row['OBJ_AMT']
                        }
                seoul_apt.append(data) # 리스트에 데이터 추가
        apt_df = pd.DataFrame(seoul_apt) # 데이터프레임으로 변환
        apt_df = apt_df.astype({'매매가(만원)':int}) # 매매가(만원) 열의 데이터 타입을 정수형으로 변환
        return apt_df

    def apt_preprocessing(self):
        apt_df = self.get_apt_data() # 아파트 실거래가 정보를 가져옴
        max_price = apt_df.groupby(['자치구명'])['매매가(만원)'].transform(max) == apt_df['매매가(만원)']
        apt_max_df = apt_df[max_price] # 자치구별로 가장 비싼 아파트 정보를 추출
        apt_max_df = apt_max_df.drop_duplicates(['건물명', '자치구명', '매매가(만원)']) # 중복 데이터 제거
        apt_max_df = apt_max_df.set_index("건물명") # 건물명을 인덱스로 설정

        # Google Maps API를 사용하여 위경도 좌표 가져오기
        gmaps_key = "YOUR_GOOGLE_MAPS_API_KEY"
        gmaps = googlemaps.Client(key=gmaps_key)

        for idx,rows in apt_max_df.iterrows():
            borough = rows["자치구명"]+idx # 자치구명과 건물명을 조합하여 주소 생성
            borough_geocode = gmaps.geocode(borough, language="ko") # 주소를 기반으로 위경도 좌표 가져오기

            # 위경도 좌표 가져오기
            borough_lat= borough_geocode[0].get("geometry")['location']['lat']
            borough_lng= borough_geocode[0].get("geometry")['location']['lng']

            apt_max_df.loc[idx, "lat"] = borough_lat # 위도 정보를 데이터프레임에 추가
            apt_max_df.loc[idx, "lng"] = borough_lng # 경도 정보를 데이터프레임에 추가
        return apt_max_df

    def get_apt_map(self):
        apt_pre_df = self.apt_preprocessing() # 아파트 실거래가 정보를 전처리
        # 서울 행정구역 json raw파일(githubcontent)
        seoul_json = req.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
        seoul_json = seoul_json.content
        seoul_json = json.loads(seoul_json)

        seoul_map = folium.Map(
            location=[37.559819, 126.963895], # 지도의 중심 좌표 설정
            zoom_start=11, # 지도의 초기 확대/축소 정도 설정
            tiles='cartodbpositron' # 지도 스타일 설정
        )

        for i in range(len(apt_pre_df)):
            folium.Marker(
                [apt_pre_df["lat"][i], apt_pre_df["lng"][i]],
                popup=f'{apt_pre_df.index[i]}' # 마커를 클릭하면 나오는 팝업 설정
                ).add_to(seoul_map) # 지도에 마커 추가

        folium.GeoJson(
            seoul_json,
            name='지역구'
        ).add_to(seoul_map) # 지도에 지역구 경계 추가

        seoul_map.save('./data/seoul_map.html') # 지도를 HTML 파일로 저장
        display(seoul_map) # 지도 출력

if __name__ == "__main__":
    apart_price = AptPrice(45) # 페이지 수를 45로 설정하여 객체 생성
    apart_price.get_apt_map() # 지도 출력하기

100%|██████████| 44/44 [00:42<00:00,  1.03it/s]
